In [86]:
import sys
!{sys.executable} -m pip install matplotlib pandas xlrd seaborn tqdm scikit-learn

    100% |████████████████████████████████| 5.4MB 2.3MB/s eta 0:00:01


In [104]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tqdm

from sklearn.cluster import KMeans

In [3]:
sales = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Продажи')
sales = sales.set_index('Material')

ap = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Магазины')
ap = ap.set_index('Material')

stock = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Остатки')
stock = stock.set_index('Material')

price = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Цены')
price = price.set_index('Material')

hierarchy = pd.read_excel('hierarchy.xlsx')

sell = pd.read_excel('Акции.xlsx')

In [88]:
sales.head()

,2014-01-01 00:00:00,2014-02-01 00:00:00,2014-03-01 00:00:00,2014-04-01 00:00:00,2014-05-01 00:00:00,2014-06-01 00:00:00,2014-07-01 00:00:00,2014-08-01 00:00:00,2014-09-01 00:00:00,2014-10-01 00:00:00,...,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00
Material,,,,,,,,,,,,,,,,,,,,,
21,22,24,52,21,11,23,20,12,8,13,...,15,21,10,10,13,9,12,8,8,6
36,0,0,0,0,0,0,0,0,0,0,...,8,18,17,9,8,7,12,2,4,1
52,0,0,0,0,0,0,0,0,0,5,...,10,17,5,18,29,19,19,13,8,14
175,0,0,0,0,0,0,0,0,0,0,...,7,8,4,3,3,5,4,3,5,5
417,18,31,69,15,8,6,14,7,9,10,...,25,34,19,29,21,33,36,22,23,15


In [91]:
corr_df = pd.DataFrame()

for x in tqdm.tqdm(sales.index[:100]):
    test_material = x

    df = pd.concat([sales.loc[test_material].rename('sales'),\
               sales.loc[test_material].shift(-1).rename('sales_1'),\
               sales.loc[test_material].shift(-2).rename('sales_2'),\
               sales.loc[test_material].shift(-3).rename('sales_3'),\
               sales.loc[test_material].shift(-4).rename('sales_4'),\
               sales.loc[test_material].shift(-5).rename('sales_5'),\
               sales.loc[test_material].shift(-6).rename('sales_6'),\
               sales.loc[test_material].shift(-9).rename('sales_9'),\
               sales.loc[test_material].shift(-12).rename('sales_12'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-1)).rename('sales_diff_1'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-2)).rename('sales_diff_2'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-3)).rename('sales_diff_3'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-4)).rename('sales_diff_4'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-5)).rename('sales_diff_5'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-6)).rename('sales_diff_6'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-9)).rename('sales_diff_9'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-12)).rename('sales_diff_12'),\
               stock.loc[test_material].rename('stock'),\
               stock.loc[test_material].shift(-1).rename('stock_1'),\
               stock.loc[test_material].shift(-2).rename('stock_2'),\
               stock.loc[test_material].shift(-3).rename('stock_3'),\
               stock.loc[test_material].shift(-4).rename('stock_4'),\
               stock.loc[test_material].subtract(stock.loc[test_material].shift(-1)).rename('stock_diff_1'),\
               stock.loc[test_material].subtract(stock.loc[test_material].shift(-2)).rename('stock_diff_2'),\
               stock.loc[test_material].subtract(stock.loc[test_material].shift(-3)).rename('stock_diff_3'),\
               stock.loc[test_material].subtract(stock.loc[test_material].shift(-4)).rename('stock_diff_4'),\
               price.loc[test_material].rename('price'),\
               price.loc[test_material].shift(-1).rename('price_1'),\
               price.loc[test_material].shift(-2).rename('price_2'),\
               price.loc[test_material].shift(-3).rename('price_3'),\
               price.loc[test_material].shift(-4).rename('price_4'),\
               price.loc[test_material].subtract(price.loc[test_material].shift(-1)).rename('price_diff_1'),\
               price.loc[test_material].subtract(price.loc[test_material].shift(-2)).rename('price_diff_2'),\
               price.loc[test_material].subtract(price.loc[test_material].shift(-3)).rename('price_diff_3'),\
               price.loc[test_material].subtract(price.loc[test_material].shift(-4)).rename('price_diff_4'),\
               ap.loc[test_material].rename('ap'),\
               ap.loc[test_material].shift(-1).rename('ap_1'),\
               ap.loc[test_material].shift(-2).rename('ap_2'),\
               ap.loc[test_material].shift(-3).rename('ap_3'),\
               ap.loc[test_material].shift(-4).rename('ap_4'),\
               ap.loc[test_material].subtract(ap.loc[test_material].shift(-1)).rename('ap_diff_1'),\
               ap.loc[test_material].subtract(ap.loc[test_material].shift(-2)).rename('ap_diff_2'),\
               ap.loc[test_material].subtract(ap.loc[test_material].shift(-3)).rename('ap_diff_3'),\
               ap.loc[test_material].subtract(ap.loc[test_material].shift(-4)).rename('ap_diff_4')],\
              axis=1)

    df.loc[:,'month'] = df.index.month

    df = df.dropna()

    corr_df = corr_df.append(df.corr().abs().sales.rename(test_material))

100%|██████████| 100/100 [00:04<00:00, 21.01it/s]


In [146]:
corr_df = pd.DataFrame()

for x in tqdm.tqdm(sales.index[:100]):
    test_material = x

    df = pd.concat([sales.loc[test_material].rename('sales'),\
               sales.loc[test_material].shift(-1).rename('sales_1'),\
               sales.loc[test_material].shift(-12).rename('sales_12'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-1)).rename('sales_diff_1'),\
               sales.loc[test_material].subtract(sales.loc[test_material].shift(-12)).rename('sales_diff_12'),\
               stock.loc[test_material].rename('stock'),\
               stock.loc[test_material].shift(-1).rename('stock_1'),\
               stock.loc[test_material].subtract(stock.loc[test_material].shift(-1)).rename('stock_diff_1'),\
               price.loc[test_material].rename('price'),\
               price.loc[test_material].shift(-1).rename('price_1'),\
               price.loc[test_material].subtract(price.loc[test_material].shift(-1)).rename('price_diff_1'),\
               ap.loc[test_material].rename('ap'),\
               ap.loc[test_material].shift(-1).rename('ap_1'),\
               ap.loc[test_material].subtract(ap.loc[test_material].shift(-1)).rename('ap_diff_1')],\
              axis=1)

    df.loc[:,'month'] = df.index.month

    df = df.dropna()

    corr_df = corr_df.append(df.corr().abs().sales.rename(test_material))

100%|██████████| 100/100 [00:02<00:00, 49.51it/s]


Убираем строки у которых недостаточная статистика по продажам, их оценивать отдельно

In [147]:
corr_df = corr_df.fillna(0)

In [154]:
#Cluster the data
kmeans = KMeans(n_clusters=6, random_state=0).fit(corr_df)
labels = kmeans.labels_

#Glue back to originaal data
corr_df['clusters'] = labels

In [155]:
cm = sns.light_palette("green", as_cmap=True)

corr_df.groupby('clusters').mean().style.background_gradient(cmap=cm)

,ap,ap_1,ap_diff_1,month,price,price_1,price_diff_1,sales,sales_1,sales_12,sales_diff_1,sales_diff_12,stock,stock_1,stock_diff_1
clusters,,,,,,,,,,,,,,,
0,0.576225,0.430818,0.355835,0.250272,0.31211,0.275334,0.100399,1,0.467417,0.231543,0.491444,0.722897,0.544435,0.445857,0.302817
1,0.824682,0.768782,0.116121,0.246425,0.693154,0.640774,0.106261,1,0.618567,0.530499,0.210225,0.231261,0.78082,0.744253,0.184423
2,0.774145,0.703187,0.254872,0.214527,0.524843,0.469228,0.115594,1,0.576018,0.162603,0.412181,0.621519,0.756646,0.71286,0.234099
3,0.463728,0.199014,0.41977,0.297419,0.272619,0.227935,0.105,1,0.149769,0.712775,0.57274,0.392993,0.499205,0.268957,0.330933
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0.518175,0.0113967,0.163571,0.156638,0.972302,0.0580201,0.676647,1,0.0480392,0.060983,0.385531,0.112098,0.389576,0.0325446,0.184716
